In [2]:
import requests
import os
import json

# Función para generar las URLs de cada año
def cargar_url():
    base_url = "https://datos.gob.es/apidata/catalog/dataset/a10002983-mediciones-por-horas-de-contaminantes-atmosfericos-y-ozono-de-la-comunitat-valenciana-"
    urls = []
    for year in range(2023, 2025):  # Recorrer de 1994 a 2024
        url = f"{base_url}{year}?_sort=title&_pageSize=10&_page=0"
        urls.append(url)
    return urls

# Función para obtener el año desde la URL principal
def obtener_anos_url(api_url):
    # Extraemos el año de la URL (suponiendo que el año es siempre el número en la URL)
    year = api_url.split('-')[-1][:4]
    return year

# Función para obtener el contenido de la URL y descargar los archivos CSV
def obtener_datos_url(api_url, year):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        response = requests.get(api_url, headers=headers)
        response.raise_for_status()  # Lanza un error si la respuesta tiene un estado diferente a 200
        
        # Obtener los datos en formato JSON
        data = response.json()  # Asume que la respuesta es JSON
        
        # Definir la carpeta de descarga según el año
        download_folder = f"{year}_csv_files"
        
        # Asegurarse de que la carpeta exista, si no, crearla
        if not os.path.exists(download_folder):
            os.makedirs(download_folder)
        
        # Recorremos todo el contenido de la respuesta buscando las URLs que terminan en .csv
        def encontrar_descargar_csv(d):
            if isinstance(d, dict):
                for key, value in d.items():
                    encontrar_descargar_csv(value)
            elif isinstance(d, list):
                for item in d:
                    encontrar_descargar_csv(item)
            elif isinstance(d, str):
                if d.endswith(".csv"):
                    print(f"URL de archivo CSV encontrado: {d}")
                    descargar_csv(d, download_folder)
        
        # Llamamos a la función para recorrer el contenido
        encontrar_descargar_csv(data)
    
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener los datos de la API: {e}")

# Función para descargar el archivo CSV
def descargar_csv(csv_url, download_folder):
    try:
        # Hacemos una solicitud GET para descargar el archivo CSV
        response = requests.get(csv_url, stream=True)
        response.raise_for_status()  # Lanza un error si la respuesta tiene un estado diferente a 200
        
        # Extraemos el nombre del archivo de la URL
        filename = csv_url.split("/")[-1]
        file_path = os.path.join(download_folder, filename)
        
        # Guardamos el archivo CSV
        with open(file_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        
        print(f"Archivo CSV descargado: {filename} en la carpeta {download_folder}")
    
    except requests.exceptions.RequestException as e:
        print(f"Error al descargar el archivo CSV desde {csv_url}: {e}")

# Recorremos todas las URLs desde 1994 a 2024
def procesar_anos():
    # Generar todas las URLs
    urls = cargar_url()
    
    # Procesamos cada URL
    for url in urls:
        year = obtener_anos_url(url)
        print(f"Procesando datos para el año {year}...")
        obtener_datos_url(url, year)

# Ejecutamos el procesamiento para todas las URLs de 1994 a 2024
procesar_anos()


Procesando datos para el año 2023...
URL de archivo CSV encontrado: https://dadesobertes.gva.es/dataset/cc91c6e4-cd97-4688-9663-b17f3db58f99/resource/019f8859-d0bb-4116-a1bb-eb7bae919f2e/download/contaminacion-atmosferica-y-ozono-promedios-por-horas_202312.csv
Archivo CSV descargado: contaminacion-atmosferica-y-ozono-promedios-por-horas_202312.csv en la carpeta 2023_csv_files
URL de archivo CSV encontrado: https://dadesobertes.gva.es/dataset/cc91c6e4-cd97-4688-9663-b17f3db58f99/resource/0c1d9611-54d4-462c-a68c-6136871e71bb/download/contaminacion-atmosferica-y-ozono-promedios-por-horas_202301.csv
Archivo CSV descargado: contaminacion-atmosferica-y-ozono-promedios-por-horas_202301.csv en la carpeta 2023_csv_files
URL de archivo CSV encontrado: https://dadesobertes.gva.es/dataset/cc91c6e4-cd97-4688-9663-b17f3db58f99/resource/7d7ba9dd-8f71-4282-9e71-6dcc3aa4c121/download/contaminacion-atmosferica-y-ozono-promedios-por-horas_202311.csv
Archivo CSV descargado: contaminacion-atmosferica-y-ozo